In [ ]:
ml_catalog = dbutils.widgets.get("ml_catalog")
ml_search_db = dbutils.widgets.get("ml_search_db")

df_click = spark.sql(f"""select * from {ml_catalog}.{ml_search_db}.ml_search_with_click
                     where action = 'actions' or action = 'people' """)

### Final Score Distribution

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [ ]:
pdf = df_click.orderBy(F.rand()).limit(10000).toPandas()
pdf['resPos'] = pdf['resPos'].astype(int)
pdf[['resPos', 'finalScore']].groupby('resPos').describe().applymap(lambda x: '%.2f' % x)

In [ ]:
pdf[['resPos', 'click']].groupby('resPos').describe().applymap(lambda x: '%.2f' % x)

### Analysis of Irrelavant Results

In [ ]:
from pathlib import Path
import sys

current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
sys.path.append(str(parent_dir))

In [ ]:
from search.utils.data_exploration import drop_docs_analysis, compare_docs_with_clicks

In [ ]:
df_click = drop_docs_analysis(df_click)

In [ ]:
# display(df_click)

In [ ]:
df_click.select('queryId').distinct().count()

In [ ]:
df_aggregated = df_click.groupBy('resPos').agg(
    F.count('keep_label').alias('startDocs'),
    F.sum(F.col('keep_label')).alias('keepDocs'),
    F.sum(F.col('click')).alias('startClicks'),
    F.sum(F.col('keep_label') * F.col('click')).alias('keepClicks')
).orderBy('resPos')

df_aggregated = (df_aggregated
.withColumn(
    'KeepDoc %', F.round(((F.col('keepDocs') / F.col('startDocs')) * 100), 2))
.withColumn(
    'KeepClicks %', 
    F.when(F.col('startClicks') == 0, 100).otherwise(F.round((F.col('keepClicks') / F.col('startClicks')) * 100, 2)))
.withColumn(
    'Start CTR %',
    F.round(F.col('startClicks') / F.col('startDocs')*100, 2))
.withColumn(
    'After CTR %',
    F.round(F.col('keepClicks') / F.col('keepDocs')*100, 2))
    )

In [ ]:
display(df_aggregated)

In [ ]:
compare_docs_with_clicks(df_aggregated)

### Extreme Cases

In [ ]:
df_dropped_clicks = (df_click
    .filter((F.col("click") == 1) & (F.col("keep_label") == 0))
    .withColumn("query", F.col("label"))  
    .withColumn("client_id", F.col("_token_client_id"))  
    .groupBy(
        F.col("resPos"),
        F.col("action"),
        F.col("query"),
        F.col("click_label"),
        F.col("client_id")
    )
    .agg(F.count("*").alias("count"))
    .orderBy(F.col("count").desc())
)

In [ ]:
display(df_dropped_clicks)

In [ ]:
collect_traceIds = (df_click
    .filter((F.col("click") == 1) & (F.col("keep_label") == 0))
    .select(F.col("action"),
            F.col("client_id"),
            F.col("traceId"), 
            F.col("resPos"),
            F.col("label"), 
            F.col("click_label"),
            F.col("finalScore")))

In [ ]:
traceIds = (collect_traceIds
    .orderBy(F.col("finalScore").desc())
    .select(F.col("traceId"))
    .collect())

In [ ]:
print(len(traceIds))

In [ ]:
trace_ids_df = spark.createDataFrame(traceIds, ["traceId"])
display(trace_ids_df)

In [ ]:
trace_ids = [row["traceId"] for row in traceIds]
# limit the number of trace_ids to 1024 to avoid exceeding the number of choices for the dropdown widget.
if len(trace_ids) > 1000:
    trace_ids = trace_ids[:1000]
default_trace_id = trace_ids[0] if len(trace_ids) > 0 else ""

In [ ]:
dbutils.widgets.dropdown(
    "trace_id",
    default_trace_id,
    trace_ids
)
trace_id = dbutils.widgets.get("trace_id")
print(f"trace_id: {trace_id}")

In [ ]:
display(spark.sql(f"""select 
                  traceId, 
                  resPos,
                  client_id,
                  label,
                  click_label,
                  click_category,
                  click_details_caption
                  from {ml_catalog}.{ml_search_db}.ml_search_with_click
                     where traceId = '{trace_id}'
                     order by resPos
                     """))

In [ ]:
display(spark.sql(f"""select 
                  traceId, 
                  resPos,
                  client_id,
                  _id,
                  caption,
                  subtitle,
                  solrScore,
                  finalScore
                  from {ml_catalog}.{ml_search_db}.ml_search_action
                     where traceId = '{trace_id}'
                     order by resPos"""))

In [ ]:
display(spark.sql(f"""select 
                  traceId, 
                  resPos,
                  client_id,
                  _id,
                  legalName, 
                  displayName, 
                  eID, 
                  location, 
                  position,
                  finalScore
                  from {ml_catalog}.{ml_search_db}.ml_search_people
                     where traceId = '{trace_id}'
                     order by resPos"""))

### Analysis of Irrelavant Results for Individual Client

In [ ]:
client_ids = [row["_token_client_id"] for row in spark.sql(f"""select distinct _token_client_id from {ml_catalog}.{ml_search_db}.ml_search_with_click
                                                           where action = 'actions' or action = 'people' """).collect()]

dbutils.widgets.dropdown(
    "client_id",
    "002",
    client_ids
)
client_id = dbutils.widgets.get("client_id")
print(f"client_id: {client_id}")

In [ ]:
df_aggregated_client = df_click.filter(F.col('_token_client_id') == client_id).groupBy('resPos').agg(
    F.count('keep_label').alias('startDocs'),
    F.sum(F.col('keep_label')).alias('keepDocs'),
    F.sum(F.col('click')).alias('startClicks'),
    F.sum(F.col('keep_label') * F.col('click')).alias('keepClicks')
).orderBy('resPos')

df_aggregated_client = (df_aggregated_client
.withColumn(
    'KeepDoc %', F.round(((F.col('keepDocs') / F.col('startDocs')) * 100), 2))
.withColumn(
    'KeepClicks %', 
    F.when(F.col('startClicks') == 0, 100).otherwise(F.round((F.col('keepClicks') / F.col('startClicks')) * 100, 2)))
.withColumn(
    'Start CTR %',
    F.round(F.col('startClicks') / F.col('startDocs')*100, 2))
.withColumn(
    'After CTR %',
    F.round(F.col('keepClicks') / F.col('keepDocs')*100, 2))
    )

In [ ]:
compare_docs_with_clicks(df_aggregated_client)

### Extreme Cases

In [ ]:
df_dropped_clicks = (df_click.filter(F.col('_token_client_id') == client_id)
    .filter((F.col("click") == 1) & (F.col("keep_label") == 0))
    .withColumn("query", F.col("label"))  
    .withColumn("client_id", F.col("_token_client_id"))  
    .groupBy(
        F.col("resPos"),
        F.col("action"),
        F.col("query"),
        F.col("click_label"),
        F.col("client_id")
    )
    .agg(F.count("*").alias("count"))
    .orderBy(F.col("count").desc())
)

In [ ]:
display(df_dropped_clicks)